# Week 3 - Segmenting and Clustering Neighborhoods in Toronto

## Homework 1

In [2]:
from bs4 import BeautifulSoup #Beautiful Soup is a Python library for pulling data out of HTML and XML files
import requests
import pandas as pd

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text

In [4]:
text_wiki = BeautifulSoup(source, 'xml')

In [5]:
table=text_wiki.find('table')

In [6]:
#dataframe with columns named: PostalCode, Borough, and Neighborhood
ncolumn = ['Postalcode','Borough','Neighborhood']
data = pd.DataFrame(columns = ncolumn)

In [7]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        data.loc[len(data)] = row_data

In [8]:
data.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
data=data[data['Borough']!='Not assigned']
data.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
data[data['Neighborhood']=='Not assigned']

,Postalcode,Borough,Neighborhood


In [11]:
## There is no cells with a borough but a Not assigned neighborhood, 

In [12]:
df=data.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df=df.reset_index(drop=False)
df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [13]:
df_final = pd.merge(data, df, on='Postalcode')

In [16]:
df_final.drop(['Neighborhood'],axis=1,inplace=True)
df_final.drop_duplicates(inplace=True)
df_final.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
df_final.head()

,Postalcode,Borough
0,M3A,North York
1,M4A,North York
2,M5A,Downtown Toronto
3,M6A,North York
4,M7A,Downtown Toronto


In [15]:
## Getting the location data
geo=pd.read_csv('http://cocl.us/Geospatial_data')
## Naming the columns for merging
geo.rename(columns={'Postal Code':'Postalcode'},inplace=True)
## Merging both dataframes
geo2 = pd.merge(geo, df_final, on='Postalcode')
## Selecting the columns for the final dataframe
geof=geo2[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
geof.head(12)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [ ]:
geof.shape